# KPI Reporting
Calculate and visualize key performance indicators (KPIs) for the operations dataset.

In [ ]:
import pandas as pd

df = pd.read_csv('../data/cleaned/orders_cleaned.csv', parse_dates=['order_date','ship_date','delivery_date'])

In [ ]:
# Monthly KPIs
df['order_month'] = df['order_date'].dt.to_period('M').dt.to_timestamp()
monthly = df.groupby('order_month').agg(
    total_orders = ('order_id','count'),
    delivered = (lambda x: (df['order_status']=='Delivered').groupby(df['order_month']).sum()),
    ontime = (lambda x: (df['is_late']==False).groupby(df['order_month']).sum())
).reset_index()
monthly['ontime_pct'] = monthly['ontime'] / monthly['delivered']
monthly.head()

In [ ]:
# Export KPIs to Excel
monthly.to_excel('../reports/monthly_kpi_report.xlsx', index=False)
print('Saved monthly KPI report to ../reports/monthly_kpi_report.xlsx')

In [ ]:
# Overall KPIs
overall = {
    'total_orders': int(df['order_id'].nunique()),
    'delivered': int((df['order_status']=='Delivered').sum()),
    'ontime_pct_overall': round(((df['order_status']=='Delivered') & (df['is_late']==False)).sum() / max(1,(df['order_status']=='Delivered').sum()),3)
}
pd.DataFrame([overall]).to_csv('../reports/overall_kpis.csv', index=False)
print('Saved overall KPI summary to ../reports/overall_kpis.csv')